In [1]:
import torch_geometric.datasets as datasets

mutag = datasets.TUDataset(root='data', name='MUTAG')

In [2]:
from gnnexplain.nn.gcn import GraphGCN
from lightning import LightningModule

model = GraphGCN.load_from_checkpoint('checkpoints/MUTAG-epoch=09.ckpt').cpu()
isinstance(model, LightningModule)

True

In [3]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch

loader = DataLoader(mutag, batch_size=40000, shuffle=False)
batch = loader.__iter__().__next__().cpu()
l = batch.to_data_list()
train, test = Batch.from_data_list(l[:140]), Batch.from_data_list(l[140:])

In [12]:
from gnnexplain.model.gtree import *
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)
gt = OptimizingExplainer(max_depth=10, max_ccp_alpha=0.02, lmbd=1e-4, n_trials=100)
gt.fit(train, model, n_jobs=1)

  0%|          | 0/100 [00:00<?, ?it/s]

Best trial: 42. Best value: 0.922671: 100%|██████████| 100/100 [00:11<00:00,  8.73it/s]


In [14]:
gt.accuracy(test)

0.8541666666666666